In [1]:
#import necessary libraries to gather and clean data
import pandas as pd
import numpy as np
from pybaseball import batting_stats
import os

In [2]:
#load data from pybaseball (takes a long time to load so lets toss the data to csv for future use)
if os.path.exists('batting-stats-2000-2024.csv'):
    stats = pd.read_csv('batting-stats-2000-2024.csv', index_col=0)
else:
    stats = batting_stats(start_season=2000, end_season=2024, qual=200)
    stats.to_csv('batting-stats-2000-2024.csv')

In [3]:
#only keep players with more than one season
stats = stats.groupby('IDfg', group_keys=False).filter(lambda x: len(x) > 1)

In [4]:
stats.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
2,1109,2001,Barry Bonds,SFG,36,153,476,664,156,49,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12.5
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
18,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.6
3,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2


In [ ]:
#create a column that shows a player's ops from the future season
def next_ops(player):
    player = player.sort_values('Season')
    player['Next_OPS'] = player['OPS'].shift(-1)
    return player

stats = stats.groupby('IDfg', group_keys=False).apply(next_ops)

In [6]:
stats[['Name','Season','OPS','Next_OPS']].head()

,Name,Season,OPS,Next_OPS
6575,Alfredo Amezaga,2006,0.664,0.682
5923,Alfredo Amezaga,2007,0.682,0.679
6214,Alfredo Amezaga,2008,0.679,NaN
2547,Garret Anderson,2000,0.827,0.792
3359,Garret Anderson,2001,0.792,0.871


In [7]:
#drop all columns with null values except our next ops column
temp = stats['Next_OPS']
stats.dropna(axis='columns', inplace=True)
stats['Next_OPS'] = temp

#also drop 'Events' because it seems that is not recorded for most players but they are given 0 instead of null
stats.drop(columns='Events', inplace=True)
stats.head()


,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,TTO%,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,L-WAR,Next_OPS
6575,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,0.217,96,98,74,97,75,42,97,1.1,0.682
5923,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,0.199,96,88,71,95,82,58,96,2.0,0.679
6214,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,0.205,99,62,81,92,86,65,100,1.2,NaN
2547,2,2000,Garret Anderson,ANA,28,159,647,681,185,107,...,0.214,104,37,82,88,117,139,93,2.2,0.792
3359,2,2001,Garret Anderson,ANA,29,161,672,704,194,125,...,0.220,108,46,86,94,111,117,102,2.7,0.871


In [8]:
#lets modify some string types to help with ml
stats.dtypes[stats.dtypes == 'object']

Name       object
Team       object
Age Rng    object
dtype: object

In [9]:
#no need for age range
stats.drop(columns='Age Rng', inplace=True)

In [10]:
#convert team name to team number
numbers, teams = stats['Team'].factorize()
stats['Team_Num'] = numbers
stats.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,L-WAR,Next_OPS,Team_Num
6575,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,96,98,74,97,75,42,97,1.1,0.682,0
5923,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,96,88,71,95,82,58,96,2.0,0.679,0
6214,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,99,62,81,92,86,65,100,1.2,NaN,0
2547,2,2000,Garret Anderson,ANA,28,159,647,681,185,107,...,104,37,82,88,117,139,93,2.2,0.792,1
3359,2,2001,Garret Anderson,ANA,29,161,672,704,194,125,...,108,46,86,94,111,117,102,2.7,0.871,1


In [11]:
#drop rows with null next_ops values (keep copy for later use)
complete_stats = stats.copy()
stats.dropna(inplace=True)

In [12]:
#import machine learning libraries / functions
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression

#higher alpha reduces overfitting, lower is more similar to linear regression (linear regression yields better results than ridge regression at least for training)
linear_regression = LinearRegression()

#split time in 3 in a chronological way
split = TimeSeriesSplit(n_splits=3)

#go through all features and find the 20 'best' features one by one
sfs = SequentialFeatureSelector(linear_regression, n_features_to_select=20, direction='forward',n_jobs=3)


In [13]:
#remove predicted value column, string columns, and general columns we do not want in the sfs
#also dropping games column because that is mostly injury luck
non_sfs_columns = ['Next_OPS','Name','Team','IDfg','Season','G']
sfs_columns = stats.columns.drop(non_sfs_columns)


In [ ]:
#scale values such that they are between 0 and 1, no negatives
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

stats.loc[:, sfs_columns] = scaler.fit_transform(stats[sfs_columns])

In [15]:
stats.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,L-WAR,Next_OPS,Team_Num
6575,1,2006,Alfredo Amezaga,FLA,0.346154,132,0.312950,0.307958,0.245690,0.278302,...,0.425532,0.212871,0.278049,0.349206,0.171233,0.117264,0.433333,0.265823,0.682,0.000000
5923,1,2007,Alfredo Amezaga,FLA,0.384615,133,0.431655,0.429066,0.323276,0.316038,...,0.425532,0.188119,0.263415,0.333333,0.219178,0.169381,0.422222,0.322785,0.679,0.000000
2547,2,2000,Garret Anderson,ANA,0.346154,159,0.875899,0.832180,0.668103,0.443396,...,0.510638,0.061881,0.317073,0.277778,0.458904,0.433225,0.388889,0.335443,0.792,0.029412
3359,2,2001,Garret Anderson,ANA,0.384615,161,0.920863,0.871972,0.706897,0.528302,...,0.553191,0.084158,0.336585,0.325397,0.417808,0.361564,0.488889,0.367089,0.871,0.029412
1396,2,2002,Garret Anderson,ANA,0.423077,158,0.859712,0.826990,0.711207,0.443396,...,0.638298,0.101485,0.273171,0.373016,0.527397,0.452769,0.522222,0.430380,0.885,0.029412


In [ ]:
#train data
sfs.fit(stats[sfs_columns], stats['Next_OPS'])

In [17]:
#return selected features aka predictors
predictor_list = list(sfs_columns[sfs.get_support()])
predictor_list

['Age',
 'AB',
 'R',
 'IBB',
 'SO',
 'SH',
 'GDP',
 'CS',
 'BB/K',
 'ISO',
 'BABIP',
 'wOBA',
 'wRAA',
 'Pos',
 'wRC+',
 'REW',
 'Def',
 'wSB',
 'BB%+',
 'K%+']

In [18]:
#train data and predict
def backtest(data, model, predictors, start=5, step=1):
    predictions = []
    years = sorted(data['Season'].unique())
    
    for i in range(start, len(years), step):
        curr_year = years[i]
        
        train = data[data['Season'] < curr_year]
        test = data[data['Season'] == curr_year]
        
        model.fit(train[predictors], train['Next_OPS'])
        
        pred = model.predict(test[predictors])
        pred = pd.Series(pred, index=test.index)
        
        combined = pd.concat([test['Next_OPS'], pred], axis=1)
        combined.columns = ['Actual', 'Prediction']
        
        predictions.append(combined)
        
    return pd.concat(predictions)

In [19]:
#return predictions
predictions = backtest(stats, linear_regression, predictor_list)
predictions

,Actual,Prediction
4971,0.756,0.752395
3085,0.694,0.726207
5732,0.645,0.723824
791,0.868,0.853943
4333,0.718,0.693636
...,...,...
5571,0.849,0.713191
5512,0.677,0.735463
1737,0.614,0.763012
1663,0.797,0.771999


In [20]:
#check accuracy of prediction (compare mean squared error with std)
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions['Actual'], predictions['Prediction'])

0.007177099358276269

In [21]:
stats['Next_OPS'].describe()

count    6644.000000
mean        0.755110
std         0.103076
min         0.398000
25%         0.686000
50%         0.748500
75%         0.816000
max         1.422000
Name: Next_OPS, dtype: float64

In [ ]:
#improve ml by using player trends by season
def player_history(df):
    df = df.sort_values('Season')
    
    df['Player_Season'] = range(0,len(df))
    df['OPS_Corr'] = list(df[['Player_Season','OPS']].expanding().corr().loc[(slice(None),'Player_Season'), 'OPS'])
    df.fillna({'OPS_Corr': 1}, inplace=True)
    
    df['OPS_Diff'] = df['OPS'] / df['OPS'].shift(1)
    df.fillna({'OPS_Diff': 1}, inplace=True)
    
    df.loc[df['OPS_Diff'] == np.inf, 'OPS_Diff'] = 1
    
    return df

stats = stats.groupby('IDfg', group_keys=False).apply(player_history)

In [23]:
def group_averages(df):
    return df['OPS'] / df['OPS'].mean()

In [ ]:
stats['OPS_Season'] = stats.groupby('Season', group_keys=False).apply(group_averages)

In [25]:
#try with new predictors
new_predictor_list = predictor_list + ['Player_Season','OPS_Corr','OPS_Season','OPS_Diff']

In [26]:
#return predictions
predictions = backtest(stats, linear_regression, new_predictor_list)

In [27]:
#check accuracy
mean_squared_error(predictions['Actual'], predictions['Prediction'])

0.007041035709579955

In [28]:
merged = predictions.merge(stats, left_index=True, right_index=True)

In [29]:
merged['Difference'] = (predictions['Actual'] - predictions['Prediction']).abs()

In [30]:
merged.head()

,Actual,Prediction,IDfg,Season,Name,Team,Age,G,AB,PA,...,ISO+,BABIP+,L-WAR,Next_OPS,Team_Num,Player_Season,OPS_Corr,OPS_Diff,OPS_Season,Difference
4971,0.756,0.764046,2,2005,Garret Anderson,LAA,0.538462,142,0.746403,0.697232,...,0.293160,0.500000,0.183544,0.756,0.058824,5,-0.411626,0.886978,0.920821,0.008046
3085,0.694,0.724491,10,2005,David Eckstein,STL,0.423077,158,0.845324,0.887543,...,0.188925,0.488889,0.354430,0.694,0.117647,4,0.036531,1.301038,0.959082,0.030491
5732,0.645,0.739116,11,2005,Darin Erstad,LAA,0.461538,153,0.807554,0.807958,...,0.185668,0.566667,0.303797,0.645,0.058824,5,-0.571292,0.862637,0.800936,0.094116
791,0.868,0.875937,15,2005,Troy Glaus,ARI,0.346154,149,0.679856,0.750865,...,0.521173,0.411111,0.398734,0.868,0.323529,5,-0.358448,0.917883,1.283028,0.007937
4333,0.718,0.713053,19,2005,Adam Kennedy,LAA,0.384615,129,0.460432,0.449827,...,0.123779,0.666667,0.398734,0.718,0.058824,5,0.354902,0.912000,0.872357,0.004947


In [31]:
merged[['IDfg', 'Season','Team', 'Name','OPS', 'Next_OPS','Prediction','Difference']].sort_values(['Difference'])

,IDfg,Season,Team,Name,OPS,Next_OPS,Prediction,Difference
3313,11615,2016,ARI,Brandon Drury,0.388462,0.764,0.763937,0.000063
5000,15518,2021,CLE,Amed Rosario,0.335577,0.715,0.715086,0.000086
2681,1736,2013,TOR,Jose Reyes,0.382692,0.726,0.726096,0.000096
7751,19293,2022,CIN,Nick Senzel,0.210577,0.696,0.695903,0.000097
5414,45,2010,- - -,Rod Barajas,0.335577,0.717,0.717126,0.000126
...,...,...,...,...,...,...,...,...
727,319,2010,WSN,Adam Dunn,0.490385,0.569,0.849726,0.280726
3881,8001,2012,- - -,Hanley Ramirez,0.362500,1.040,0.752072,0.287928
2106,5310,2010,CHC,Tyler Colvin,0.417308,0.509,0.809130,0.300130
5038,96,2007,ATL,Andruw Jones,0.328846,0.505,0.809228,0.304228


In [32]:
#get data ready for data visualization (want all the normal stats + prediction and next ops)
merge_stats = predictions.merge(complete_stats, left_index=True, right_index=True)

In [33]:
#Actual and Next_OPS are the same so only keep one
merge_stats.drop(columns='Actual')

,Prediction,IDfg,Season,Name,Team,Age,G,AB,PA,H,...,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,L-WAR,Next_OPS,Team_Num
4971,0.764046,2,2005,Garret Anderson,LAA,33,142,575,603,163,...,106,49,88,93,102,96,103,-0.2,0.756,2
3085,0.724491,10,2005,David Eckstein,STL,30,158,630,713,185,...,109,93,38,107,92,64,102,2.5,0.694,4
5732,0.739116,11,2005,Darin Erstad,LAA,31,153,609,667,166,...,102,90,104,98,87,63,109,1.7,0.645,2
791,0.875937,15,2005,Troy Glaus,ARI,28,149,538,634,139,...,96,151,142,107,122,166,95,3.2,0.868,11
4333,0.713053,19,2005,Adam Kennedy,LAA,29,129,416,460,125,...,112,80,88,107,87,44,118,3.2,0.718,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,0.712862,27815,2023,Jordan Westburg,BAL,24,68,208,228,54,...,105,84,106,98,98,87,113,1.1,0.849,27
5512,0.727892,29622,2023,Sal Frelick,MIL,23,57,191,223,47,...,98,142,74,105,84,63,96,1.4,0.677,20
1737,0.772231,29766,2023,Zack Gelof,OAK,23,69,270,300,72,...,108,104,118,106,122,144,112,2.9,0.614,12
1663,0.761920,30116,2023,Seiya Suzuki,CHC,28,138,515,583,147,...,114,115,100,110,117,120,115,3.2,0.797,21


In [34]:
#convert data to csv for data visualization
merge_stats.to_csv('OPS-Prediction-2000-2023.csv')